# Infer-8-Model-Selection : Selection et Comparaison de Modeles

**Serie** : Programmation Probabiliste avec Infer.NET (8/13)  
**Duree estimee** : 45 minutes  
**Prerequis** : Infer-7-Classification

---

## Objectifs

- Comprendre le probleme du surapprentissage
- Calculer l'evidence du modele (marginal likelihood)
- Utiliser le facteur de Bayes pour comparer des modeles
- Implementer l'Automatic Relevance Determination (ARD)

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-7-Classification](Infer-7-Classification.ipynb) | [Infer-9-Topic-Models](Infer-9-Topic-Models.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour explorer la selection de modeles bayesienne. Cette approche permet de comparer objectivement differents modeles en calculant leur evidence marginale, implementant ainsi le rasoir d'Occam de maniere mathematique.

In [2]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [3]:
// Chargement du helper pour visualisation des graphes de facteurs
#load "FactorGraphHelper.cs"

Console.WriteLine($"FactorGraphHelper charge. Graphviz disponible : {FactorGraphHelper.IsGraphvizAvailable()}");

FactorGraphHelper charge. Graphviz disponible : True


### Visualisation des graphes de facteurs

Ce notebook utilise `FactorGraphHelper.cs` pour visualiser les graphes de facteurs generes par Infer.NET. Ces graphes montrent la structure probabiliste des modeles : les noeuds representent les variables aleatoires et les observations, les facteurs (carres) representent les distributions conditionnelles.

Pour activer la visualisation, on configure `ShowFactorGraph = true` sur le moteur d'inference.

### Note technique : Calcul de l'evidence dans Infer.NET

Infer.NET calcule l'evidence du modele via une astuce elegante :

1. On introduit une variable indicatrice `evidence = Bernoulli(0.5)`
2. Le modele est conditionne par `Variable.If(evidence)`
3. Apres inference, `evidence.LogOdds` donne le log de l'evidence

> **Pourquoi ca fonctionne ?** Par le theoreme de Bayes :
> $$P(\text{evidence}=\text{true}|D) \propto P(D|\text{evidence}=\text{true}) \times P(\text{evidence}=\text{true})$$
> 
> Comme $P(\text{evidence}=\text{true}) = 0.5$, le log-odds posterieur est directement le log de l'evidence (a une constante pres).

Cette methode est specifique a Infer.NET et permet d'obtenir l'evidence sans calcul integral explicite.

## 2. Le Probleme du Surapprentissage

### Observation

Un modele complexe peut parfaitement ajuster les donnees d'entrainement mais mal generaliser.

### Exemple

Ajuster un polynome de degre n-1 a n points : ajustement parfait mais prediction catastrophique.

### Solution bayesienne

- Les priors penalisent les modeles complexes
- L'evidence du modele equilibre ajustement et complexite
- C'est le **rasoir d'Occam bayesien**

## 3. Evidence du Modele (Marginal Likelihood)

### Definition

$$P(D|M) = \int P(D|\theta, M) P(\theta|M) d\theta$$

L'evidence est la probabilite des donnees sous le modele, marginalisee sur les parametres.

### Interpretation

- Un modele simple fait des predictions moins precises mais moins dispersees
- Un modele complexe fait des predictions plus precises mais plus dispersees
- L'evidence favorise le bon equilibre

In [4]:
// Calcul de l'evidence avec Infer.NET

// Donnees
double[] observations = { 13, 15, 17, 14, 16, 15, 18 };
int n = observations.Length;

// MODELE 1 : Une seule gaussienne
Variable<bool> evidence1 = Variable.Bernoulli(0.5).Named("evidence1");

using (Variable.If(evidence1))
{
    Variable<double> moyenne1 = Variable.GaussianFromMeanAndPrecision(15, 0.01).Named("moyenne");
    Variable<double> precision1 = Variable.GammaFromShapeAndScale(2, 0.5).Named("precision");
    
    for (int i = 0; i < n; i++)
    {
        Variable<double> obs1 = Variable.GaussianFromMeanAndPrecision(moyenne1, precision1).Named($"obs_{i}");
        obs1.ObservedValue = observations[i];
    }
}

InferenceEngine moteur1 = new InferenceEngine();
moteur1.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur1.ShowFactorGraph = true;  // Activation de la visualisation

double logEvidence1 = moteur1.Infer<Bernoulli>(evidence1).LogOdds;

Console.WriteLine("=== Evidence du Modele ===");
Console.WriteLine($"\nModele 1 (1 gaussienne) : log evidence = {logEvidence1:F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Evidence du Modele ===

Modele 1 (1 gaussienne) : log evidence = -16,98


### Lecture du resultat

**Log evidence = -16.98** pour le modele a une gaussienne.

Cette valeur negative est normale : c'est un logarithme de probabilite, donc toujours negatif (ou nul). Plus la valeur est proche de 0, meilleure est l'evidence.

En termes absolus, $e^{-16.98} \approx 4.2 \times 10^{-8}$ semble tres petit, mais c'est la **comparaison relative** entre modeles qui importe, pas la valeur absolue.

In [5]:
// Visualisation du graphe de facteurs - Modele 1 gaussienne
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_23_40_66.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 evidence1 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 15 
 
<!-- node4 -->
 
 node4 
 
 Gaussian 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 moyenne 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 0,01 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 precision 
 
<!-- node11 -->
 
 node11 
 
 Gaussian 
 
<!-- node6->node11 -->
 
 node6->node11 
 
 
 mean 
 
<!-- node13 -->
 
 node13 
 
 Gaussian 
 
<!-- node6->node13 -->
 
 node6->node13 
 
 
 mean 
 
<!-- node15 -->
 
 node15 
 
 Gaussian 
 
<!-- node6->node15 -->
 
 node6->node15 
 
 
 mean 
 
<!-- node17 -->
 
 node17 
 
 Gaussian 
 
<!-- node6->node17 -->
 
 node6->node17 
 
 
 mean 
 
<!-- node19 -->
 
 node19 
 
 Gaussian 
 
<!-- node6->node19 -->
 
 node6->node19 
 
 
 mean 
 
<!-- node21 -->
 
 node21 
 
 Gaussian 
 
<!-- node6->node21 -->
 
 node6->node21 
 
 
 mean 
 
<!-- node23 -->
 
 node23 
 
 Gaussian 
 
<!-- node6->node23 -->
 
 node6->node23 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 2 
 
<!-- node8 -->
 
 node8 
 
 Sample 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 shape 
 
<!-- node10 -->
 
 node10 
 
 precision 
 
<!-- node8->node10 -->
 
 node8->node10 
 
 
 
<!-- node9 -->
 
 node9 
 
 0,5 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 scale 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 precision 
 
<!-- node10->node13 -->
 
 node10->node13 
 
 
 precision 
 
<!-- node10->node15 -->
 
 node10->node15 
 
 
 precision 
 
<!-- node10->node17 -->
 
 node10->node17 
 
 
 precision 
 
<!-- node10->node19 -->
 
 node10->node19 
 
 
 precision 
 
<!-- node10->node21 -->
 
 node10->node21 
 
 
 precision 
 
<!-- node10->node23 -->
 
 node10->node23 
 
 
 precision 
 
<!-- node12 -->
 
 node12 
 
 13 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 
<!-- node14 -->
 
 node14 
 
 15 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 
<!-- node16 -->
 
 node16 
 
 17 
 
<!-- node15->node16 -->
 
 node15->node16 
 
 
 
<!-- node18 -->
 
 node18 
 
 14 
 
<!-- node17->node18 -->
 
 node17->node18 
 
 
 
<!-- node20 -->
 
 node20 
 
 16 
 
<!-- node19->node20 -->
 
 node19->node20 
 
 
 
<!-- node22 -->
 
 node22 
 
 15 
 
<!-- node21->node22 -->
 
 node21->node22 
 
 
 
<!-- node24 -->
 
 node24 
 
 18 
 
<!-- node23->node24 -->
 
 node23->node24


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Modele 1 gaussienne

Le graphe montre la structure du modele a une gaussienne :

- **Noeuds ovales** : variables aleatoires (`moyenne`, `precision`, `evidence1`)
- **Noeuds carres** : facteurs (distributions conditionnelles)
- **Noeuds gris** : observations (`obs_0` a `obs_6`)

La variable `evidence1` (Bernoulli) englobe tout le modele via `Variable.If()`. Les 7 observations partagent la meme `moyenne` et `precision`, ce qui represente l'hypothese i.i.d. (independantes et identiquement distribuees).

### Implementation : Modele 2 - Melange de deux gaussiennes

Le **modele de melange** (mixture model) suppose que chaque observation provient de l'une ou l'autre de deux gaussiennes, avec une probabilite $\pi$ pour la premiere et $1-\pi$ pour la seconde.

**Parametres du modele** :
- $\mu_1, \mu_2$ : moyennes des deux composantes
- $\tau$ : precision commune (simplification)
- $\pi$ : proportion du melange (poids de la premiere composante)

**Code** : Pour chaque observation, on tire d'abord `composante ~ Bernoulli(pi)`, puis on observe depuis la gaussienne correspondante.

> **Note algorithmique** : Nous utilisons `VariationalMessagePassing` car les melanges de gaussiennes sont plus stables avec VMP qu'avec EP pour le calcul d'evidence.

In [6]:
// MODELE 2 : Melange de deux gaussiennes
Variable<bool> evidence2 = Variable.Bernoulli(0.5).Named("evidence2");

using (Variable.If(evidence2))
{
    Variable<double> moyenne2a = Variable.GaussianFromMeanAndPrecision(10, 0.01).Named("moyenne_a");
    Variable<double> moyenne2b = Variable.GaussianFromMeanAndPrecision(20, 0.01).Named("moyenne_b");
    Variable<double> precision2 = Variable.GammaFromShapeAndScale(2, 0.5).Named("precision");
    Variable<double> poidsMixte = Variable.Beta(1, 1).Named("poids_mixte");
    
    for (int i = 0; i < n; i++)
    {
        Variable<bool> composante = Variable.Bernoulli(poidsMixte).Named($"comp_{i}");
        Variable<double> obs2 = Variable.New<double>().Named($"obs2_{i}");
        using (Variable.If(composante))
        {
            obs2.SetTo(Variable.GaussianFromMeanAndPrecision(moyenne2a, precision2));
        }
        using (Variable.IfNot(composante))
        {
            obs2.SetTo(Variable.GaussianFromMeanAndPrecision(moyenne2b, precision2));
        }
        obs2.ObservedValue = observations[i];
    }
}

InferenceEngine moteur2 = new InferenceEngine(new VariationalMessagePassing());
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur2.ShowFactorGraph = true;  // Activation de la visualisation

double logEvidence2 = moteur2.Infer<Bernoulli>(evidence2).LogOdds;

Console.WriteLine($"Modele 2 (melange 2 gaussiennes) : log evidence = {logEvidence2:F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Modele 2 (melange 2 gaussiennes) : log evidence = -20,12


In [7]:
// Visualisation du graphe de facteurs - Modele melange
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_23_42_78.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 evidence2 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 10 
 
<!-- node4 -->
 
 node4 
 
 Gaussian 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 moyenne_a 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 0,01 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 precision 
 
<!-- node20 -->
 
 node20 
 
 Gaussian 
 
<!-- node6->node20 -->
 
 node6->node20 
 
 
 mean 
 
<!-- node25 -->
 
 node25 
 
 Gaussian 
 
<!-- node6->node25 -->
 
 node6->node25 
 
 
 mean 
 
<!-- node30 -->
 
 node30 
 
 Gaussian 
 
<!-- node6->node30 -->
 
 node6->node30 
 
 
 mean 
 
<!-- node35 -->
 
 node35 
 
 Gaussian 
 
<!-- node6->node35 -->
 
 node6->node35 
 
 
 mean 
 
<!-- node40 -->
 
 node40 
 
 Gaussian 
 
<!-- node6->node40 -->
 
 node6->node40 
 
 
 mean 
 
<!-- node45 -->
 
 node45 
 
 Gaussian 
 
<!-- node6->node45 -->
 
 node6->node45 
 
 
 mean 
 
<!-- node50 -->
 
 node50 
 
 Gaussian 
 
<!-- node6->node50 -->
 
 node6->node50 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 20 
 
<!-- node8 -->
 
 node8 
 
 Gaussian 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node10 -->
 
 node10 
 
 moyenne_b 
 
<!-- node8->node10 -->
 
 node8->node10 
 
 
 
<!-- node9 -->
 
 node9 
 
 0,01 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 precision 
 
<!-- node22 -->
 
 node22 
 
 Gaussian 
 
<!-- node10->node22 -->
 
 node10->node22 
 
 
 mean 
 
<!-- node27 -->
 
 node27 
 
 Gaussian 
 
<!-- node10->node27 -->
 
 node10->node27 
 
 
 mean 
 
<!-- node32 -->
 
 node32 
 
 Gaussian 
 
<!-- node10->node32 -->
 
 node10->node32 
 
 
 mean 
 
<!-- node37 -->
 
 node37 
 
 Gaussian 
 
<!-- node10->node37 -->
 
 node10->node37 
 
 
 mean 
 
<!-- node42 -->
 
 node42 
 
 Gaussian 
 
<!-- node10->node42 -->
 
 node10->node42 
 
 
 mean 
 
<!-- node47 -->
 
 node47 
 
 Gaussian 
 
<!-- node10->node47 -->
 
 node10->node47 
 
 
 mean 
 
<!-- node52 -->
 
 node52 
 
 Gaussian 
 
<!-- node10->node52 -->
 
 node10->node52 
 
 
 mean 
 
<!-- node11 -->
 
 node11 
 
 2 
 
<!-- node12 -->
 
 node12 
 
 Sample 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 shape 
 
<!-- node14 -->
 
 node14 
 
 precision 
 
<!-- node12->node14 -->
 
 node12->node14 
 
 
 
<!-- node13 -->
 
 node13 
 
 0,5 
 
<!-- node13->node12 -->
 
 node13->node12 
 
 
 scale 
 
<!-- node14->node20 -->
 
 node14->node20 
 
 
 precision 
 
<!-- node14->node22 -->
 
 node14->node22 
 
 
 precision 
 
<!-- node14->node25 -->
 
 node14->node25 
 
 
 precision 
 
<!-- node14->node27 -->
 
 node14->node27 
 
 
 precision 
 
<!-- node14->node30 -->
 
 node14->node30 
 
 
 precision 
 
<!-- node14->node32 -->
 
 node14->node32 
 
 
 precision 
 
<!-- node14->node35 -->
 
 node14->node35 
 
 
 precision 
 
<!-- node14->node37 -->
 
 node14->node37 
 
 
 precision 
 
<!-- node14->node40 -->
 
 node14->node40 
 
 
 precision 
 
<!-- node14->node42 -->
 
 node14->node42 
 
 
 precision 
 
<!-- node14->node45 -->
 
 node14->node45 
 
 
 precision 
 
<!-- node14->node47 -->
 
 node14->node47 
 
 
 precision 
 
<!-- node14->node50


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Modele melange

Le graphe du modele de melange est plus complexe :

- **moyenne_a, moyenne_b** : les deux centres des composantes gaussiennes
- **poids_mixte** : parametre Beta controlant la proportion du melange
- **comp_i** : variable latente discrete (quelle composante genere l'observation i ?)
- **obs2_i** : observations, chacune connectee aux deux moyennes via sa variable de composante

La structure en "gateway" (Variable.If/IfNot) cree des branchements conditionnels visibles dans le graphe. Chaque observation peut provenir de l'une ou l'autre gaussienne selon `comp_i`.

## 4. Facteur de Bayes

### Definition

$$BF_{12} = \frac{P(D|M_1)}{P(D|M_2)} = \exp(\log E_1 - \log E_2)$$

### Interpretation (echelle de Jeffreys)

| log(BF) | BF | Evidence pour M1 |
|---------|----|-----------------|
| 0-1 | 1-3 | Negligeable |
| 1-2 | 3-10 | Substantielle |
| 2-3 | 10-30 | Forte |
| 3-5 | 30-150 | Tres forte |
| >5 | >150 | Decisive |

In [8]:
// Facteur de Bayes
double logBF = logEvidence1 - logEvidence2;
double BF = Math.Exp(logBF);

Console.WriteLine("=== Facteur de Bayes ===");
Console.WriteLine($"\nlog(BF) = {logBF:F2}");
Console.WriteLine($"BF = {BF:F2}");

string interpretation;
if (Math.Abs(logBF) < 1) interpretation = "Evidence negligeable";
else if (Math.Abs(logBF) < 2) interpretation = "Evidence substantielle";
else if (Math.Abs(logBF) < 3) interpretation = "Evidence forte";
else interpretation = "Evidence tres forte/decisive";

string favori = logBF > 0 ? "Modele 1 (1 gaussienne)" : "Modele 2 (melange)";
Console.WriteLine($"\n{interpretation} en faveur de : {favori}");

=== Facteur de Bayes ===

log(BF) = 3,14
BF = 23,03

Evidence tres forte/decisive en faveur de : Modele 1 (1 gaussienne)


### Interprétation du facteur de Bayes

**Résultat** : log(BF) = 3.14, BF ≈ 23

Selon l'échelle de Jeffreys, un BF de 23 représente une **evidence forte** (entre 10 et 30) en faveur du modèle 1.

**Pourquoi le modèle simple gagne-t-il ?**

Les données {13, 15, 17, 14, 16, 15, 18} sont **unimodales** avec moyenne ~15.3. Le modèle à 2 gaussiennes :
1. Introduit des paramètres inutiles (2 moyennes, poids du mélange)
2. Ces paramètres doivent être "expliqués" par le prior
3. Le prior "dilue" la vraisemblance sur un espace plus grand

C'est le **rasoir d'Occam bayésien** en action : à ajustement égal, le modèle simple est préféré car il fait des prédictions plus "concentrées".

> **Formule intuitive** : Evidence ≈ (vraisemblance) × (volume du prior utilisé) / (volume total du prior)

## 5. Selection du Nombre de Composantes

### Application

Determiner le nombre optimal de composantes dans un modele de melange.

### Donnees bimodales : quand le modele complexe gagne

Les donnees precedentes etaient unimodales (autour de 15). Testons maintenant avec des donnees **clairement bimodales** : deux groupes bien separes autour de 6 et 15.

**Question** : Le modele a 2 composantes va-t-il maintenant etre prefere ?

L'algorithme compare systematiquement 1 vs 2 composantes en calculant l'evidence de chaque modele.

In [9]:
// Donnees bimodales
double[] dataBimodal = { 5, 6, 7, 5.5, 6.5, 15, 16, 17, 14, 15.5, 16.5, 6, 15 };
int nBi = dataBimodal.Length;

Console.WriteLine("=== Selection du nombre de composantes ===");
Console.WriteLine($"Donnees : {string.Join(", ", dataBimodal)}\n");

// Test avec 1, 2, 3 composantes
double[] logEvidences = new double[3];

// 1 composante
{
    Variable<bool> ev = Variable.Bernoulli(0.5);
    using (Variable.If(ev))
    {
        Variable<double> m = Variable.GaussianFromMeanAndPrecision(10, 0.01);
        Variable<double> p = Variable.GammaFromShapeAndScale(2, 0.5);
        foreach (var d in dataBimodal)
        {
            Variable<double> o = Variable.GaussianFromMeanAndPrecision(m, p);
            o.ObservedValue = d;
        }
    }
    var eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    logEvidences[0] = eng.Infer<Bernoulli>(ev).LogOdds;
}

Console.WriteLine($"1 composante : log evidence = {logEvidences[0]:F2}");

// 2 composantes - simplifie
{
    Variable<bool> ev = Variable.Bernoulli(0.5);
    using (Variable.If(ev))
    {
        Variable<double> m1 = Variable.GaussianFromMeanAndPrecision(6, 0.1);
        Variable<double> m2 = Variable.GaussianFromMeanAndPrecision(15, 0.1);
        Variable<double> p = Variable.GammaFromShapeAndScale(2, 1);
        Variable<double> w = Variable.Beta(1, 1);
        
        foreach (var d in dataBimodal)
        {
            Variable<bool> c = Variable.Bernoulli(w);
            Variable<double> o = Variable.New<double>();
            using (Variable.If(c)) { o.SetTo(Variable.GaussianFromMeanAndPrecision(m1, p)); }
            using (Variable.IfNot(c)) { o.SetTo(Variable.GaussianFromMeanAndPrecision(m2, p)); }
            o.ObservedValue = d;
        }
    }
    var eng = new InferenceEngine(new VariationalMessagePassing());
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    logEvidences[1] = eng.Infer<Bernoulli>(ev).LogOdds;
}

Console.WriteLine($"2 composantes : log evidence = {logEvidences[1]:F2}");

// Meilleur modele
int meilleur = logEvidences[0] > logEvidences[1] ? 1 : 2;
Console.WriteLine($"\n=> Le modele a {meilleur} composante(s) est prefere");

=== Selection du nombre de composantes ===
Donnees : 5, 6, 7, 5,5, 6,5, 15, 16, 17, 14, 15,5, 16,5, 6, 15

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
1 composante : log evidence = -45,89
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
2 composantes : log evidence = -31,35

=> Le modele a 2 composante(s) est prefere


### Analyse des resultats : donnees bimodales

**Donnees** : deux groupes distincts autour de 6 et 15

| Modele | Log evidence | Interpretation |
|--------|--------------|----------------|
| 1 composante | -45.89 | Mal adapte (moyenne ~10.5 ne represente aucun groupe) |
| 2 composantes | -31.35 | Bien adapte (capture les deux modes) |

**Difference** : log(BF) = -31.35 - (-45.89) = 14.54

Un facteur de Bayes $e^{14.54} \approx 2 \times 10^6$ en faveur du modele a 2 composantes constitue une evidence **decisive**.

> **Lecon cle** : Le facteur de Bayes detecte automatiquement la structure des donnees. Il prefere le modele simple quand les donnees sont simples (section 3-4) et le modele complexe quand les donnees l'exigent (ici).

## 6. Automatic Relevance Determination (ARD)

### Principe

ARD utilise des priors hierarchiques pour determiner automatiquement quelles features sont pertinentes.

### Modele

$$\alpha_f \sim \text{Gamma}(a, b)$$
$$w_f \sim \mathcal{N}(0, \alpha_f^{-1})$$

Si $\alpha_f$ devient grand, le poids $w_f$ est contraint pres de 0 -> feature non pertinente.

### Transition : de la comparaison de modeles a la selection de features

Jusqu'ici, nous avons compare des **modeles entiers** (1 vs 2 gaussiennes, lineaire vs quadratique). Mais en pratique, on veut souvent repondre a une question plus fine :

**Quelles features sont vraiment utiles dans mon modele ?**

C'est le probleme de la **selection de variables**. L'approche bayesienne offre une solution elegante : l'**Automatic Relevance Determination** (ARD).

### Generation des donnees synthetiques

Nous creons un probleme de regression ou :
- **Feature 1** : coefficient reel = 2.0 (pertinente)
- **Feature 2** : coefficient reel = 0.0 (non pertinente)
- **Feature 3** : coefficient reel = 3.0 (pertinente)

Le modele ARD devra "decouvrir" automatiquement que la feature 2 n'apporte aucune information predictive.

In [10]:
// ARD pour regression

// Donnees : y = 2*x1 + 0*x2 + 3*x3 + bruit
// x2 est une feature non pertinente
int nSamples = 20;
int nFeatures = 3;
Random rng = new Random(42);

double[,] X = new double[nSamples, nFeatures];
double[] y = new double[nSamples];
double[] vraisPoids = { 2.0, 0.0, 3.0 };  // x2 a poids 0

for (int i = 0; i < nSamples; i++)
{
    for (int f = 0; f < nFeatures; f++)
    {
        X[i, f] = rng.NextDouble() * 2 - 1;  // [-1, 1]
    }
    y[i] = vraisPoids[0] * X[i, 0] + vraisPoids[1] * X[i, 1] + vraisPoids[2] * X[i, 2]
           + rng.NextDouble() * 0.5 - 0.25;  // Bruit
}

Console.WriteLine("=== ARD : Automatic Relevance Determination ===");
Console.WriteLine($"\nVrais poids : w1={vraisPoids[0]}, w2={vraisPoids[1]} (non pertinent), w3={vraisPoids[2]}");

=== ARD : Automatic Relevance Determination ===

Vrais poids : w1=2, w2=0 (non pertinent), w3=3


### Construction du modele ARD hierarchique

Le modele ARD introduit un **hyperparametre de precision** $\alpha_f$ pour chaque feature $f$ :

$$w_f \sim \mathcal{N}(0, \alpha_f^{-1})$$

**Interpretation** :
- Si $\alpha_f$ est petit (ex: 0.3), la variance $1/\alpha_f$ est grande, donc $w_f$ peut prendre des valeurs significatives
- Si $\alpha_f$ devient grand (ex: 10), la variance est petite, $w_f$ est "pousse" vers 0

Les $\alpha_f$ sont eux-memes des variables aleatoires avec prior `Gamma(1, 1)`. L'inference determine simultanement les poids et leur pertinence.

In [11]:
// Modele ARD
Range sampleRange = new Range(nSamples).Named("sample");
Range featureRange = new Range(nFeatures).Named("feature");

// Precisions par feature (ARD)
VariableArray<double> alpha = Variable.Array<double>(featureRange).Named("alpha");
alpha[featureRange] = Variable.GammaFromShapeAndScale(1, 1).ForEach(featureRange);

// Poids avec prior dependant de alpha
VariableArray<double> poids = Variable.Array<double>(featureRange).Named("poids");
using (Variable.ForEach(featureRange))
{
    poids[featureRange] = Variable.GaussianFromMeanAndPrecision(0, alpha[featureRange]);
}

// Bruit de l'observation
Variable<double> noisePrecision = Variable.GammaFromShapeAndScale(2, 1).Named("noise");

// Donnees
VariableArray2D<double> xVar = Variable.Array<double>(sampleRange, featureRange).Named("x");
VariableArray<double> yVar = Variable.Array<double>(sampleRange).Named("y");

using (Variable.ForEach(sampleRange))
{
    Variable<double> prediction = Variable.Constant(0.0);
    for (int f = 0; f < nFeatures; f++)
    {
        prediction = prediction + poids[f] * xVar[sampleRange, f];
    }
    yVar[sampleRange] = Variable.GaussianFromMeanAndPrecision(prediction, noisePrecision);
}

xVar.ObservedValue = X;
yVar.ObservedValue = y;

InferenceEngine moteurARD = new InferenceEngine(new ExpectationPropagation());
moteurARD.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurARD.ShowFactorGraph = true;  // Activation de la visualisation

Gaussian[] poidsPost = moteurARD.Infer<Gaussian[]>(poids);
Gamma[] alphaPost = moteurARD.Infer<Gamma[]>(alpha);

Console.WriteLine("\nResultats ARD :");
for (int f = 0; f < nFeatures; f++)
{
    double wMean = poidsPost[f].GetMean();
    double wStd = Math.Sqrt(poidsPost[f].GetVariance());
    double alphaMean = alphaPost[f].GetMean();
    string relevance = alphaMean > 5 ? "faible" : alphaMean > 1 ? "moyenne" : "haute";
    Console.WriteLine($"  Feature {f+1} : poids = {wMean:F2} +/- {wStd:F2}, alpha = {alphaMean:F2} (pertinence {relevance})");
}

Console.WriteLine("\n=> Les features avec alpha eleve sont considerees non pertinentes");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50

Resultats ARD :
  Feature 1 : poids = 2,04 +/- 0,12, alpha = 0,49 (pertinence haute)
  Feature 2 : poids = 0,02 +/- 0,11, alpha = 1,49 (pertinence moyenne)
  Feature 3 : poids = 2,97 +/- 0,15, alpha = 0,28 (pertinence haute)

=> Les features avec alpha eleve sont considerees non pertinentes


In [ ]:
// Visualisation du graphe de facteurs - Modele ARD
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

### Lecture du graphe de facteurs - Modele ARD hierarchique

Le graphe ARD illustre la structure hierarchique a deux niveaux :

**Niveau hyperparametres** :
- **alpha[feature]** : precision par feature (hyperparametre Gamma)
- Le prior sur `alpha` controle la "force" de la regularisation

**Niveau parametres** :
- **poids[feature]** : poids de regression, chacun avec une precision `alpha[f]` differente
- Les poids sont couples : `poids[f] ~ N(0, 1/alpha[f])`

**Niveau donnees** :
- **x[sample, feature]** : matrice des features (observee)
- **y[sample]** : cible (observee)
- **noise** : precision du bruit (infere)

La structure "plate notation" avec `ForEach` cree des repetitions sur les ranges `sample` et `feature`, representees par des boites dans le graphe.

### Analyse des résultats ARD

**Comparaison vrais poids vs estimations** :

| Feature | Vrai poids | Estimé | α estimé | Pertinence |
|---------|------------|--------|----------|------------|
| 1 | **2.0** | 2.04 | 0.49 | Haute ✓ |
| 2 | **0.0** | 0.02 | 1.49 | Moyenne |
| 3 | **3.0** | 2.97 | 0.28 | Haute ✓ |

**Interprétation des hyperparamètres α** :
- **α petit** (< 1) → le prior sur w est large → w peut prendre des valeurs significatives → feature pertinente
- **α grand** (> 1) → le prior sur w est concentré autour de 0 → w contraint à ~0 → feature non pertinente

**Pourquoi Feature 2 n'a pas α très élevé ?**

Avec seulement 20 échantillons, le modèle reste incertain. α = 1.49 indique une pertinence "moyenne" plutôt que clairement nulle. Plus de données augmenteraient α pour cette feature.

> **Application pratique** : ARD est utilisé en machine learning pour la **sélection automatique de features** sans validation croisée explicite.

## 7. Validation Croisee Bayesienne

### Principe

Au lieu de diviser les donnees, utiliser la predictive posterieure pour evaluer le modele.

### Leave-One-Out (LOO)

$$\text{LOO-CV} = \sum_{i=1}^n \log P(y_i | y_{-i}, M)$$

### Algorithme Leave-One-Out bayesien

Pour chaque point $i$ :
1. **Entrainer** le modele sur tous les points sauf $i$
2. **Calculer** la distribution predictive posterieure
3. **Evaluer** la log-probabilite du point $i$ sous cette predictive

La **variance predictive** combine deux sources d'incertitude :
- L'incertitude sur la moyenne ($\text{Var}(\mu)$)
- Le bruit inherent des observations ($1/\tau$)

$$\sigma^2_{\text{pred}} = \text{Var}(\mu) + \frac{1}{\mathbb{E}[\tau]}$$

In [12]:
// Validation LOO simplifiee

double[] dataLOO = { 10, 12, 11, 13, 12, 11, 14, 10, 12, 11 };
int nLOO = dataLOO.Length;

double totalLogPred = 0;

for (int i = 0; i < nLOO; i++)
{
    // Entrainer sur toutes les donnees sauf i
    Variable<double> mu = Variable.GaussianFromMeanAndPrecision(10, 0.01);
    Variable<double> prec = Variable.GammaFromShapeAndScale(2, 0.5);
    
    for (int j = 0; j < nLOO; j++)
    {
        if (j != i)
        {
            Variable<double> obs = Variable.GaussianFromMeanAndPrecision(mu, prec);
            obs.ObservedValue = dataLOO[j];
        }
    }
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Gaussian muPost = eng.Infer<Gaussian>(mu);
    Gamma precPost = eng.Infer<Gamma>(prec);
    
    // Probabilite predictive pour le point i
    double predMean = muPost.GetMean();
    double predVar = muPost.GetVariance() + 1.0 / precPost.GetMean();
    
    double logProb = Gaussian.FromMeanAndVariance(predMean, predVar).GetLogProb(dataLOO[i]);
    totalLogPred += logProb;
}

Console.WriteLine("=== Validation Leave-One-Out ===");
Console.WriteLine($"\nLog predictive totale : {totalLogPred:F2}");
Console.WriteLine($"Log predictive moyenne : {totalLogPred / nLOO:F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Validation Leave-One-Out ===

Log predictive totale : -18,13
Log predictive moyenne : -1,81


### Interpretation de la validation LOO

**Log predictive moyenne = -1.81**

Cette metrique mesure la capacite du modele a predire des observations non vues. Comparons avec des references :

| Log predictive moyenne | Qualite du modele |
|------------------------|-------------------|
| > -1.0 | Excellente |
| -1.0 a -2.0 | Bonne |
| -2.0 a -3.0 | Acceptable |
| < -3.0 | Mauvaise |

Notre valeur de -1.81 indique une **bonne** capacite predictive.

> **Avantage du LOO bayesien** : Contrairement au LOO classique, cette methode :
> - Utilise l'incertitude complete (pas juste une estimation ponctuelle)
> - Tient compte de l'incertitude sur les parametres via la variance predictive
> - Ne necessite pas de reentrainer completement le modele (en theorie, via des approximations)

## 8. Exercice : Comparer Polynomes

### Enonce

Comparez trois modeles de regression :
- Lineaire : y = a*x + b
- Quadratique : y = a*x^2 + b*x + c
- Cubique : y = a*x^3 + b*x^2 + c*x + d

Sur des donnees lineaires avec bruit.

### Mise en pratique : comparaison lineaire vs quadratique

Les donnees sont generees selon $y = 2x + 1 + \epsilon$ (relation lineaire).

**Modeles a comparer** :
- **Lineaire** : $y = ax + b$ (2 parametres)
- **Quadratique** : $y = ax^2 + bx + c$ (3 parametres)

Le modele quadratique peut representer la relation lineaire (avec $a \approx 0$), mais paie un "cout de complexite" pour ce parametre inutile.

> **Hint** : Pour ajouter un modele cubique, il suffirait d'ajouter un terme $d \times x^3$. Le meme raisonnement s'applique : plus de parametres = plus de penalite si ils ne sont pas necessaires.

In [13]:
// EXERCICE : Comparaison de modeles polynomiaux

// Donnees lineaires : y = 2*x + 1 + bruit
double[] xPoly = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 };
double[] yPoly = { 1.2, 3.1, 4.8, 7.2, 8.9, 11.1, 13.0, 14.8, 17.2, 19.1 };
int nPoly = xPoly.Length;

Console.WriteLine("=== Comparaison de Modeles Polynomiaux ===");
Console.WriteLine("Vraie relation : y = 2*x + 1\n");

// Modele lineaire
Variable<bool> evLin = Variable.Bernoulli(0.5).Named("evidence_lin");
using (Variable.If(evLin))
{
    Variable<double> a = Variable.GaussianFromMeanAndVariance(0, 10).Named("a_lin");
    Variable<double> b = Variable.GaussianFromMeanAndVariance(0, 10).Named("b_lin");
    Variable<double> noise = Variable.GammaFromShapeAndScale(2, 0.5).Named("noise_lin");
    
    for (int i = 0; i < nPoly; i++)
    {
        Variable<double> pred = (a * xPoly[i] + b).Named($"pred_lin_{i}");
        Variable<double> obs = Variable.GaussianFromMeanAndPrecision(pred, noise).Named($"obs_lin_{i}");
        obs.ObservedValue = yPoly[i];
    }
}
var engLin = new InferenceEngine();
engLin.Compiler.CompilerChoice = CompilerChoice.Roslyn;
engLin.ShowFactorGraph = true;  // Activation de la visualisation
double logEvLin = engLin.Infer<Bernoulli>(evLin).LogOdds;
Console.WriteLine($"Modele lineaire : log evidence = {logEvLin:F2}");

// Modele quadratique
Variable<bool> evQuad = Variable.Bernoulli(0.5).Named("evidence_quad");
using (Variable.If(evQuad))
{
    Variable<double> a = Variable.GaussianFromMeanAndVariance(0, 10).Named("a_quad");
    Variable<double> b = Variable.GaussianFromMeanAndVariance(0, 10).Named("b_quad");
    Variable<double> c = Variable.GaussianFromMeanAndVariance(0, 10).Named("c_quad");
    Variable<double> noise = Variable.GammaFromShapeAndScale(2, 0.5).Named("noise_quad");
    
    for (int i = 0; i < nPoly; i++)
    {
        Variable<double> pred = (a * xPoly[i] * xPoly[i] + b * xPoly[i] + c).Named($"pred_quad_{i}");
        Variable<double> obs = Variable.GaussianFromMeanAndPrecision(pred, noise).Named($"obs_quad_{i}");
        obs.ObservedValue = yPoly[i];
    }
}
var engQuad = new InferenceEngine();
engQuad.Compiler.CompilerChoice = CompilerChoice.Roslyn;
engQuad.ShowFactorGraph = true;  // Activation de la visualisation
double logEvQuad = engQuad.Infer<Bernoulli>(evQuad).LogOdds;
Console.WriteLine($"Modele quadratique : log evidence = {logEvQuad:F2}");

// Meilleur modele
string meilleurMod = logEvLin > logEvQuad ? "Lineaire" : "Quadratique";
Console.WriteLine($"\n=> Le modele {meilleurMod} est prefere (rasoir d'Occam)");

=== Comparaison de Modeles Polynomiaux ===
Vraie relation : y = 2*x + 1

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Modele lineaire : log evidence = -14,03
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Modele quadratique : log evidence = -20,28

=> Le modele Lineaire est prefere (rasoir d'Occam)


In [14]:
// Visualisation du graphe de facteurs - Modele quadratique (dernier compile)
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_24_13_18.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 evidence_quad 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node3 -->
 
 node3 
 
 0 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 a_quad 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 10 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node19 -->
 
 node19 
 
 Multiply 
 
<!-- node6->node19 -->
 
 node6->node19 
 
 
 a 
 
<!-- node34 -->
 
 node34 
 
 Multiply 
 
<!-- node6->node34 -->
 
 node6->node34 
 
 
 a 
 
<!-- node49 -->
 
 node49 
 
 Multiply 
 
<!-- node6->node49 -->
 
 node6->node49 
 
 
 a 
 
<!-- node64 -->
 
 node64 
 
 Multiply 
 
<!-- node6->node64 -->
 
 node6->node64 
 
 
 a 
 
<!-- node79 -->
 
 node79 
 
 Multiply 
 
<!-- node6->node79 -->
 
 node6->node79 
 
 
 a 
 
<!-- node94 -->
 
 node94 
 
 Multiply 
 
<!-- node6->node94 -->
 
 node6->node94 
 
 
 a 
 
<!-- node109 -->
 
 node109 
 
 Multiply 
 
<!-- node6->node109 -->
 
 node6->node109 
 
 
 a 
 
<!-- node124 -->
 
 node124 
 
 Multiply 
 
<!-- node6->node124 -->
 
 node6->node124 
 
 
 a 
 
<!-- node139 -->
 
 node139 
 
 Multiply 
 
<!-- node6->node139 -->
 
 node6->node139 
 
 
 a 
 
<!-- node154 -->
 
 node154 
 
 Multiply 
 
<!-- node6->node154 -->
 
 node6->node154 
 
 
 a 
 
<!-- node7 -->
 
 node7 
 
 0 
 
<!-- node8 -->
 
 node8 
 
 GaussianFromMeanAndVariance 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node10 -->
 
 node10 
 
 b_quad 
 
<!-- node8->node10 -->
 
 node8->node10 
 
 
 
<!-- node9 -->
 
 node9 
 
 10 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 variance 
 
<!-- node25 -->
 
 node25 
 
 Multiply 
 
<!-- node10->node25 -->
 
 node10->node25 
 
 
 a 
 
<!-- node40 -->
 
 node40 
 
 Multiply 
 
<!-- node10->node40 -->
 
 node10->node40 
 
 
 a 
 
<!-- node55 -->
 
 node55 
 
 Multiply 
 
<!-- node10->node55 -->
 
 node10->node55 
 
 
 a 
 
<!-- node70 -->
 
 node70 
 
 Multiply 
 
<!-- node10->node70 -->
 
 node10->node70 
 
 
 a 
 
<!-- node85 -->
 
 node85 
 
 Multiply 
 
<!-- node10->node85 -->
 
 node10->node85 
 
 
 a 
 
<!-- node100 -->
 
 node100 
 
 Multiply 
 
<!-- node10->node100 -->
 
 node10->node100 
 
 
 a 
 
<!-- node115 -->
 
 node115 
 
 Multiply 
 
<!-- node10->node115 -->
 
 node10->node115 
 
 
 a 
 
<!-- node130 -->
 
 node130 
 
 Multiply 
 
<!-- node10->node130 -->
 
 node10->node130 
 
 
 a 
 
<!-- node145 -->
 
 node145 
 
 Multiply 
 
<!-- node10->node145 -->
 
 node10->node145 
 
 
 a 
 
<!-- node160 -->
 
 node160 
 
 Multiply 
 
<!-- node10->node160 -->
 
 node10->node160 
 
 
 a 
 
<!-- node11 -->
 
 node11 
 
 0 
 
<!-- node12 -->
 
 node12 
 
 GaussianFromMeanAndVariance 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 mean 
 
<!-- node14 -->
 
 node14 
 
 c_quad 
 
<!-- node12->node14 -->
 
 node12->node14 
 
 
 
<!-- node13 -->
 
 node13 
 
 10 
 
<!-- node13->node12 -->
 
 node13->node12 
 
 
 variance 
 
<!-- node30 -->
 
 node30 
 
 Plus 
 
<!-- node14->node30 -->
 
 node14->node30 
 
 
 b 
 
<!-- node45 -->
 
 node45 
 
 Plus 
 
<!--


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs - Modele quadratique

Le graphe montre la structure du modele quadratique $y = ax^2 + bx + c$ :

- **a_quad, b_quad, c_quad** : les trois coefficients du polynome (priors Gaussiens)
- **noise_quad** : precision du bruit d'observation (prior Gamma)
- **evidence_quad** : variable indicatrice pour le calcul d'evidence
- **pred_quad_i** : predictions intermediaires (combinaison des coefficients)
- **obs_quad_i** : observations (valeurs fixees)

Comparativement au modele lineaire, ce graphe contient un parametre supplementaire (`c_quad`), ce qui augmente la complexite du modele et la "surface" du prior - d'ou la penalisation par le rasoir d'Occam bayesien.

### Analyse de l'exercice polynomes

**Resultats** :

| Modele | Parametres | Log evidence |
|--------|------------|--------------|
| Lineaire | a, b (2) | -14.03 |
| Quadratique | a, b, c (3) | -20.28 |

**Facteur de Bayes** : $\exp(-14.03 - (-20.28)) = \exp(6.25) \approx 518$

Le modele lineaire est **decisement** prefere (BF > 150).

**Pourquoi le modele quadratique perd-il ?**

1. **Donnees generees lineairement** : y = 2x + 1 + bruit
2. **Coefficient quadratique inutile** : le parametre $a$ (coefficient de $x^2$) n'apporte rien
3. **Penalite de complexite** : le prior sur $a$ "dilue" la vraisemblance

> **Exercice supplementaire** : Que se passerait-il si les donnees etaient generees par y = 0.1*x^2 + 2*x + 1 ? Le terme quadratique est present mais faible. Le modele lineaire pourrait encore gagner si le signal quadratique est noye dans le bruit - c'est la balance ajustement vs complexite en action.

## 9. Resume

| Concept | Description |
|---------|-------------|
| **Evidence** | P(D\|M) - vraisemblance marginale |
| **Facteur de Bayes** | Ratio d'evidences pour comparer modeles |
| **Rasoir d'Occam** | Preference automatique pour modeles simples |
| **ARD** | Selection automatique de features |
| **LOO-CV** | Validation sans diviser les donnees |

---

## Prochaine etape

Dans [Infer-9-Topic-Models](Infer-9-Topic-Models.ipynb), nous explorerons :

- Latent Dirichlet Allocation (LDA)
- Modelisation de topics dans les documents
- Inference sur structures hierarchiques complexes

---

## Annexe : Distributions et concepts

### Distributions utilisees dans ce notebook

| Distribution | Role | Parametres typiques |
|--------------|------|---------------------|
| `Bernoulli(0.5)` | Variable indicatrice pour calcul d'evidence | p=0.5 (prior non informatif) |
| `GaussianFromMeanAndPrecision` | Modele d'observation | mean~15, precision~1 |
| `GammaFromShapeAndScale` | Prior sur precision | shape=2, scale=0.5 |
| `Beta(1,1)` | Prior sur proportion de melange | Prior uniforme sur [0,1] |

### Concepts probabilistes illustres

| Concept | Section | Application |
|---------|---------|-------------|
| Evidence marginale $P(D\|M)$ | 3, 5 | Comparer modeles sans validation croisee |
| Facteur de Bayes | 4 | Quantifier la preference pour un modele |
| Rasoir d'Occam bayesien | 3-5 | Penalisation automatique de la complexite |
| Priors hierarchiques | 6 | ARD pour selection de features |
| Distribution predictive | 7 | LOO-CV bayesien |

### Applications pratiques

- **Selection du nombre de composantes** : Clustering avec nombre de clusters inconnu
- **Selection de features** : Regression avec beaucoup de covariables
- **Choix d'architecture** : Reseaux bayesiens avec structure variable
- **Comparaison de familles** : Lineaire vs non-lineaire, parametrique vs non-parametrique

### Points cles a retenir

**1. Le rasoir d'Occam est automatique**

La selection de modeles bayesienne penalise naturellement la complexite. Pas besoin de criteres ad hoc comme l'AIC ou le BIC - l'evidence marginale fait le travail.

**2. L'echelle compte**

| Methode | Quand l'utiliser |
|---------|------------------|
| Facteur de Bayes | Comparer 2-3 modeles distincts |
| ARD | Selectionner parmi de nombreuses features |
| LOO-CV | Evaluer la capacite predictive |

**3. Les priors sont importants**

Les priors vagues ($\sigma^2 = 10$ sur les poids) penalisent moins que des priors informatifs. Un mauvais choix de prior peut fausser la comparaison.

> **Conseil pratique** : Pour une comparaison equitable, utilisez des priors de meme "force" (meme variance) sur les parametres comparables entre modeles.